# A Note on this Notebook
This can be run as either a python/ipython script or as a notebook.
It generates a firedrake `DumbCheckpoint` file called `true-fields.h5` containing the values of $u_{true}$ and $q_{true}$ in `Function`s named `u_true` and `q_true` respectively.
The investigation continues in another notebook which uses these fields.

# Problem Description

We try to enforce posterior consistency in the non-point-cloud case by redefining our objective functional

$$J''[u, q] = 
\underbrace{ N \int_{\Omega}\left(u_{interpolated} - u\right)^2dx}_{\text{model-data misfit}} + 
\underbrace{\alpha^2\int_\Omega|\nabla q|^2dx}_{\text{regularization}}$$

which is the same as $J'$ but where $J''_{\text{misfit}} = N \times J'_{\text{misfit}}$ to try to allow the misfit term to grow with number of measurements.

$\alpha = 0.02$ is used throughout.

# Setup

In [ ]:
from scipy.interpolate import (
    LinearNDInterpolator,
    NearestNDInterpolator,
    CloughTocher2DInterpolator,
    Rbf,
)

import matplotlib.pyplot as plt
import firedrake
import firedrake_adjoint

from firedrake import Constant, cos, sin

import numpy as np
from numpy import pi as π
from numpy import random

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

import os

currentdir = os.path.dirname(os.path.realpath('__file__'))

In [ ]:
mesh = firedrake.UnitSquareMesh(32, 32)

# Solution Space
V = firedrake.FunctionSpace(mesh, family='CG', degree=2)

# q (Control) Space
Q = firedrake.FunctionSpace(mesh, family='CG', degree=2)

## Fake $q_{true}$

In [ ]:
seed = 1729
generator = random.default_rng(seed)

degree = 5
x = firedrake.SpatialCoordinate(mesh)

q_true = firedrake.Function(Q, name='q_true')
for k in range(degree):
    for l in range(int(np.sqrt(degree**2 - k**2))):
        Z = np.sqrt(1 + k**2 + l**2)
        ϕ = 2 * π * (k * x[0] + l * x[1])

        A_kl = generator.standard_normal() / Z
        B_kl = generator.standard_normal() / Z

        expr = Constant(A_kl) * cos(ϕ) + Constant(B_kl) * sin(ϕ)
        mode = firedrake.interpolate(expr, Q)

        q_true += mode

print('Made fake q_true')

## Fake $u_{true}$

In [ ]:
from firedrake import exp, inner, grad, dx
u_true = firedrake.Function(V, name='u_true')
v = firedrake.TestFunction(V)
f = Constant(1.0)
k0 = Constant(0.5)
bc = firedrake.DirichletBC(V, 0, 'on_boundary')
F = (k0 * exp(q_true) * inner(grad(u_true), grad(v)) - f * v) * dx
firedrake.solve(F == 0, u_true, bc)

print('Made fake u_true')

In [ ]:
# Clear tape since don't need to have taped above
tape = firedrake_adjoint.get_working_tape()
tape.clear_tape()

## Data Output
We save our fields to a firedrake checkpoint file.

In [ ]:
filename = os.path.join(currentdir, 'true-fields')

with firedrake.DumbCheckpoint(filename, mode=firedrake.FILE_CREATE) as chk:
    chk.store(q_true)
    chk.store(u_true)

Make sure they have saved...

In [ ]:
with firedrake.DumbCheckpoint(filename, mode=firedrake.FILE_READ) as chk:
    chk.load(q_true, name='q_true')
    chk.load(u_true, name='u_true')

In [ ]:
fig, axes = plt.subplots()
axes.set_aspect('equal')
colors = firedrake.tripcolor(q_true, axes=axes, shading='gouraud')
fig.colorbar(colors);

In [ ]:
fig, axes = plt.subplots()
axes.set_aspect('equal')
colors = firedrake.tripcolor(u_true, axes=axes, shading='gouraud')
fig.colorbar(colors);